In [ ]:
from openai import OpenAI
import os

In [ ]:
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(base_url="https://openrouter.ai/api/v1"
                       ,api_key=openai_api_key
)
print("Successfully connected to OpenRouter")
print(openai_api_key[:5])

In [ ]:
from IPython.display import display, Markdown

def print_markdown(text):
    display(Markdown(text))

In [ ]:
from PIL import Image

image_filename = "images/sample.jpg"
img = Image.open(image_filename)
print(f"Image {image_filename} loaded successfully")
print(f"Format: {img.format}, Size: {img.size}")
print(f"Mode: {img.mode}")

image_to_analyze = img


In [ ]:
import io
import base64

In [ ]:
def encode_image_to_base64(image_path_or_pil):
    if isinstance(image_path_or_pil, str):  # If it's a file path
        # Check if the file exists
        if not os.path.exists(image_path_or_pil):
            raise FileNotFoundError(f"Image file not found at: {image_path_or_pil}")
        with open(image_path_or_pil, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")

    elif isinstance(image_path_or_pil, Image.Image):
        buffer = io.BytesIO()
        image_format = image_path_or_pil.format or "JPEG"
        image_path_or_pil.save(buffer, format=image_format)
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError("Input must be a file path (str) or a PIL Image object.")


In [ ]:
def query_openai_vision(client, image, prompt, model="qwen/qwen2.5-vl-72b-instruct:free", max_tokens=1000):

    base64_image = encode_image_to_base64(image)

    try:
        messages_to_send = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                    },
                ],
            }
        ]

        response = client.chat.completions.create(
            model = model,
            messages = messages_to_send,
            max_tokens = max_tokens,
        )
        return response.choices[0].message.content

    except Exception as e:
        return f"Error calling API: {e}"

In [ ]:
food_recognition_prompt = """
# Nutritional Analysis Task

## Context
You are a nutrition expert analyzing food images to provide accurate nutritional information.

## Instructions
Analyze the food item in the image and provide estimated nutritional information based on your knowledge.

## Input
- An image of a food item

## Output
Provide the following estimated nutritional information for a typical serving size or per 100g:
- food_name (string)
- serving_description (string, e.g., '1 slice', '100g', '1 cup')
- calories (float)
- fat_grams (float)
- protein_grams (float)
- confidence_level (string: 'High', 'Medium', or 'Low')

**IMPORTANT:** give brief description related to the health benefits of each serving in the picture provided.

"""

In [ ]:
print("Querying OpenAI Vision API...")
openai_desciption = query_openai_vision(
    openai_client,
    image_to_analyze,
    food_recognition_prompt,
)
print_markdown(openai_desciption)
display(image_to_analyze)